In [1]:
!git clone https://github.com/OpenGVLab/InternVL.git

fatal: destination path 'InternVL' already exists and is not an empty directory.


In [2]:
!cd pretrained/

/bin/bash: line 1: cd: pretrained/: No such file or directory


In [3]:
# !huggingface-cli download --resume-download --local-dir-use-symlinks False OpenGVLab/InternVL2_5-1B-MPO --local-dir InternVL2_5-1B-MPO

# Dataset Creation

In [4]:
!pip install datasets

In [5]:
# hugging face dataset download
from datasets import load_dataset
dataset = load_dataset("your_dataset_apth", token="input_token", split='train[:100]')

In [6]:

dataset

Dataset({
    features: ['image', 'question', 'answer'],
    num_rows: 100
})

In [7]:
# dataset["image"]

In [8]:
import os
from PIL import Image
import requests
from io import BytesIO

# Step 1: Create dataset folder and image folder
dataset_folder = "dataset_folder"
image_folder = os.path.join(dataset_folder, "images")

os.makedirs(image_folder, exist_ok=True)

# Step 2: Save images to the 'images' folder
for idx, sample in enumerate(dataset):
    # Assuming dataset has an 'image' column with PIL Image or URL
    if isinstance(sample['image'], Image.Image):
        image = sample['image']
    else:
        # If it's a URL, download the image
        response = requests.get(sample['image']['url'])
        image = Image.open(BytesIO(response.content))

    # Save the image
    image_path = os.path.join(image_folder, f"image_{idx}.jpg")
    image.save(image_path)
    # print(f"Saved: {image_path}")

print("All images have been saved successfully!")


All images have been saved successfully!


In [9]:
final_dataset_json = []
relative_image_path = "/content/dataset_folder/images"
for idx, sample in enumerate(dataset):
    temp_json_dataset = {}

    x, y = dataset["image"][idx].size
    # Assuming dataset has an 'image' column with PIL Image or URL
    # print(sample["question"], sample["answer"])


    temp_json_dataset["id"] = idx
    temp_json_dataset["image"] = f"{relative_image_path}/image_{idx}.jpg"
    temp_json_dataset["width"] = x
    temp_json_dataset["height"] = y
    temp_json_dataset["conversations"] = [
      {
        "from": "human",
        "value": f"<image>\n{sample['question']}"
      },
      {
        "from": "gpt",
        "value": f"{sample['answer']}"
      }
    ]
    final_dataset_json.append(temp_json_dataset)

    # if idx == 2:
    #   break

    # break

In this format json file required
```
{
  "id": 0,
  "image": "images/00000000.jpg",
  "width": 897,
  "height": 1152,
  "conversations": [
    {
      "from": "human",
      "value": "<image>\nCan you extract any readable text from the image?"
    },
    {
      "from": "gpt",
      "value": "Dares Wins Vol. 5 Tommy's Heroes Vol. 6: For Tomorrow Vol. 7: Closing Time miniseries. Clark Kent is being interviewed about Superman's connection to notorious killer Tommy Monaghan. Taking the conversation..."
    }
  ]
}
```

In [10]:
# final_dataset_json

In [11]:
import json
with open('data.json', 'w') as jsonl_file:
    for item in final_dataset_json:
        # Convert the dictionary to a JSON string
        json_line = json.dumps(item)

        # Write the JSON string followed by a newline
        jsonl_file.write(json_line + '\n')

In [12]:
# creating meta_data_file for data information
import json

meta_data = {
    "data": {
        "root": "/content/dataset_folder/images",
        "annotation": "/content/data.json",
        "data_augment": False,
        "repeat_time": 1,
        "length": len(dataset)
    }
}
with open("/content/InternVL/internvl_chat/shell/data/train_metadata.json", "w") as f:
    json.dump(meta_data, f)

In [13]:
import sys
import os

# Get the current script's directory or set a custom path
# current_path = os.path.abspath("/content/InternVL")

# # Add the directory to sys.path
# if current_path not in sys.path:
#     sys.path.append(current_path)

sys.path.append("/content/InternVL")

In [14]:
os.chdir('/content/InternVL/internvl_chat')

In [15]:
!pwd

/content/InternVL/internvl_chat


In [16]:
# !pip install flash-attn==2.0.2

In [17]:
!pip install -r /content/InternVL/requirements/internvl_chat.txt

  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.42.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached peft-0.10.0-py3-none-any.whl.metadata (13 kB)
Using cached bitsandbytes-0.42.0-py3-none-any.whl (105.0 MB)
Using cached peft-0.10.0-py3-none-any.whl (199 kB)
Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.45.2
    Uninstalling bitsandbytes-0.45.2:
      Successfully uninstalled bitsandbytes-0.45.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.4.0.dev0
    Uninstalling accelerate-1.4.0.dev0:
      Successfully uninstalled accelerate-1.4.0.dev0
  Attempting uninstall: peft
    Found existing installation: peft 0.14.1.dev0
    Uninstalling peft-0.14.1.dev0:
      Successfully uninstalled peft-0.14.1.dev0


In [18]:
!pip install deepspeed

In [19]:
# # !pip install flash_attn exllamav2 transforme  r -U
# !pip install psutil
# !pip install flash_attn --no-build-isolation

In [20]:
!nvidia-smi

Thu Feb 20 05:11:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [21]:
# !GPUS=1 PER_DEVICE_BATCH_SIZE=1 sh /content/InternVL/internvl_chat/shell/internvl2.5_mpo/preference_optimization/internvl2_5_1b_qwen2_5_0_5b_dynamic_res_mpo.sh

In [22]:
!pip install flash_attn exllamav2 -U

In [23]:
# !pip install --upgrade transformers

In [24]:
# !pip uninstall flash-attn
# !pip install flash-attn --no-build-isolation --no-cache-dir

In [25]:
import torch
from transformers import AutoTokenizer, AutoModel
path = "OpenGVLab/InternVL2_5-1B"
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval().cuda()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possibl

In [26]:
import transformers
print(transformers.__version__)

4.37.2


In [27]:
!sed -i 's#\./shell/data/internvl_1_2_finetune_custom.json#/content/InternVL/internvl_chat/shell/data/train_metadata.json#g' /content/InternVL/internvl_chat/shell/internvl2.5/2nd_finetune/internvl2_5_1b_dynamic_res_2nd_finetune_lora.sh


In [28]:
!GPUS=1 PER_DEVICE_BATCH_SIZE=1 sh /content/InternVL/internvl_chat/shell/internvl2.5/2nd_finetune/internvl2_5_1b_dynamic_res_2nd_finetune_lora.sh

+ GPUS=1
+ BATCH_SIZE=16
+ PER_DEVICE_BATCH_SIZE=1
+ GRADIENT_ACC=16
+ pwd
+ export PYTHONPATH=/env/python:/content/InternVL/internvl_chat
+ export MASTER_PORT=34229
+ export TF_CPP_MIN_LOG_LEVEL=3
+ export LAUNCHER=pytorch
+ OUTPUT_DIR=work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora
+ [ ! -d work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora ]
+ tee -a work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora/training_log.txt
+ torchrun --nnodes=1 --node_rank=0 --master_addr=127.0.0.1 --nproc_per_node=1 --master_port=34229 internvl/train/internvl_chat_finetune.py --model_name_or_path OpenGVLab/InternVL2_5-1B --conv_style internvl2_5 --use_fast_tokenizer False --output_dir work_dirs/internvl_chat_v2_5/internvl2_5_1b_dynamic_res_2nd_finetune_lora --meta_path /content/InternVL/internvl_chat/shell/data/train_metadata.json --overwrite_output_dir True --force_image_size 448 --max_dynamic_patch 6 --down_sample_ratio 0.5 --drop_

In [29]:
# from transformers.models.llama.modeling_llama import LLAMA_ATTENTION_CLASSES

In [30]:
import transformers.models.llama.modeling_llama

In [31]:
import inspect
import transformers.models.llama.modeling_llama as llama

# List all functions
functions = [name for name, obj in inspect.getmembers(llama, inspect.isfunction)]
print("📘 Functions:", functions)

# List all classes
classes = [name for name, obj in inspect.getmembers(llama, inspect.isclass)]
print("🏛️ Classes:", classes)


📘 Functions: ['_expand_mask', '_get_unpad_data', '_make_causal_mask', '_prepare_4d_attention_mask', '_prepare_4d_causal_attention_mask', '_prepare_4d_causal_attention_mask_for_sdpa', 'add_start_docstrings', 'add_start_docstrings_to_model_forward', 'apply_rotary_pos_emb', 'flash_attn_func', 'flash_attn_varlen_func', 'is_flash_attn_2_available', 'is_flash_attn_greater_or_equal_2_10', 'is_torch_fx_available', 'pad_input', 'repeat_kv', 'replace_return_docstrings', 'rotate_half', 'unpad_input']
🏛️ Classes: ['AttentionMaskConverter', 'BCEWithLogitsLoss', 'BaseModelOutputWithPast', 'Cache', 'CausalLMOutputWithPast', 'CrossEntropyLoss', 'DynamicCache', 'LlamaAttention', 'LlamaConfig', 'LlamaDecoderLayer', 'LlamaDynamicNTKScalingRotaryEmbedding', 'LlamaFlashAttention2', 'LlamaForCausalLM', 'LlamaForSequenceClassification', 'LlamaLinearScalingRotaryEmbedding', 'LlamaMLP', 'LlamaModel', 'LlamaPreTrainedModel', 'LlamaRMSNorm', 'LlamaRotaryEmbedding', 'LlamaSdpaAttention', 'MSELoss', 'PreTrainedMod

In [32]:
!pip freeze

absl-py==1.4.0
accelerate==0.34.2
aiohappyeyeballs==2.4.6
aiohttp==3.11.12
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.4
ale-py==0.10.1
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.6.0
arviz==0.20.0
astropy==7.0.1
astropy-iers-data==0.2025.2.10.0.33.26
astunparse==1.6.3
atpublic==4.1.0
attrs==25.1.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
beautifulsoup4==4.13.3
betterproto==2.0.0b6
bigframes==1.36.0
bigquery-magics==0.5.0
bitsandbytes==0.42.0
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==3.1.0
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.1
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.88
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.20.0
cloudpickle==3.1.1
cmake==3.31.4
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confe